In [33]:
%jars ../../Sources/*

## Imports

In [34]:
import calculador.Calculador;
import calculador.Pair;

import frsf.cidisi.faia.agent.Action;
import frsf.cidisi.faia.agent.Agent;
import frsf.cidisi.faia.agent.Perception;

import frsf.cidisi.faia.agent.search.GoalTest;
import frsf.cidisi.faia.agent.search.Problem;
import frsf.cidisi.faia.agent.search.SearchAction;
import frsf.cidisi.faia.agent.search.SearchBasedAgent;
import frsf.cidisi.faia.agent.search.SearchBasedAgentState;

import frsf.cidisi.faia.environment.Environment;

import frsf.cidisi.faia.exceptions.PrologConnectorException;

import frsf.cidisi.faia.simulator.SearchBasedAgentSimulator;

import frsf.cidisi.faia.solver.search.*;

import frsf.cidisi.faia.state.AgentState;
import frsf.cidisi.faia.state.EnvironmentState;

import java.awt.Point;
import java.util.Iterator;
import java.util.logging.Level;
import java.util.logging.Logger;
import java.util.Vector;

# Agent Definition

## Snake States

In [35]:
public class EstadoSnake extends SearchBasedAgentState {

    private Vector<Point> composicionDelAgente;
    /*
     * Orientación
     * 
     * 0 Norte
     * 1 Sur
     * 2 Este
     * 3 Oeste
     */
    private int orientacionDeLaCabeza;
    public static int NORTE = 0;
    public static int SUR = 1;
    public static int ESTE = 2;
    public static int OESTE = 3;
    /*
     * Mundo Conocido
     * 
     * 0 No hay nada
     * 1 Límite del mundo
     * 2 Comida
     * 3 Parte del Agente
     * -1 Desconocido
     */
    /*
     * El mundoConocido es un vector que tiene las filas que adentro tiene un vector que indica las columnas.
     */
    private Vector<Vector<Integer>> mundoConocido;
    private int longitud;
    private boolean estaVivo;
    private Vector<Point> celdasVisitadas;
    private double costo;
    private boolean encontreElLimiteAlNorte;
    private boolean encontreElLimiteAlSur;
    private boolean encontreElLimiteAlEste;
    private boolean encontreElLimiteAlOeste;
    private int vecesQueSeGiro;
    private int vecesQueSeGiroDerecha;
    public static int NO_HAY_NADA = 0;
    public static int LIMITE_DEL_MUNDO = 1;
    public static int COMIDA = 2;
    public static int PARTE_DEL_AGENTE = 3;
    public static int DESCONOCIDO = -1;

    public EstadoSnake() {
        this.initState();
    }

    public void updateState(Perception p) {
        // Actualizar las percepciones
        Point cabeza = this.composicionDelAgente.firstElement();

        // Al Norte de la cabeza
        if (((PercepcionSnake) p).getSensorNorte() == PercepcionSnake.LIMITE_DEL_MUNDO) {
            this.encontreElLimiteAlNorte = true;

            // Vemos si hay fila de desconocidos al norte
            boolean hayFilaDeDesconocidos = false;

            for (int col = 0; col < this.mundoConocido.size(); col++) {
                hayFilaDeDesconocidos = (this.mundoConocido.firstElement().get(col) == PercepcionSnake.DESCONOCIDO);

                if (hayFilaDeDesconocidos == false) {
                    break;
                }
            }

            if (this.mundoConocido.size() == 10) {
//				if (!hayFilaDeDesconocidos) {
//					return;
//				}
//				else {
                if (hayFilaDeDesconocidos) {
                    boolean columnaDeDesconocidosAlOeste = false;
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        columnaDeDesconocidosAlOeste = (this.mundoConocido.get(fil).firstElement() == PercepcionSnake.DESCONOCIDO);

                        if (!columnaDeDesconocidosAlOeste) {
                            break;
                        }
                    }

                    boolean columnaDeDesconocidosAlEste = false;
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        columnaDeDesconocidosAlEste = (this.mundoConocido.get(fil).lastElement() == PercepcionSnake.DESCONOCIDO);

                        if (!columnaDeDesconocidosAlEste) {
                            break;
                        }
                    }

                    this.mundoConocido.removeElementAt(0);
                    while (hayFilaDeDesconocidos) {

                        if (columnaDeDesconocidosAlOeste) {
                            for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                                this.mundoConocido.get(fil).removeElementAt(0);
                            //						Actualizo la posición del agente
                            }
                            Iterator<Point> it = this.composicionDelAgente.iterator();
                            while (it.hasNext()) {
                                Point par = it.next();
                                par.setLocation(par.getX(), par.getY() - 1);
                            }

                            // Actualizo la posición de las celdas visitadas
                            it = this.celdasVisitadas.iterator();
                            while (it.hasNext()) {
                                Point par = it.next();
                                par.setLocation(par.getX(), par.getY() - 1);
                            }

                            for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                                columnaDeDesconocidosAlOeste = (this.mundoConocido.get(fil).firstElement() == PercepcionSnake.DESCONOCIDO);

                                if (!columnaDeDesconocidosAlOeste) {
                                    break;
                                }
                            }
                        }
                        if (columnaDeDesconocidosAlEste) {
                            for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                                this.mundoConocido.get(fil).removeElementAt(this.mundoConocido.size() - 1);
                            }
                            for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                                columnaDeDesconocidosAlEste = (this.mundoConocido.get(fil).lastElement() == PercepcionSnake.DESCONOCIDO);

                                if (!columnaDeDesconocidosAlEste) {
                                    break;
                                }
                            }
                        }

                        Vector<Integer> filaDeDesconocidos = new Vector<Integer>();

                        for (int col = 0; col < this.mundoConocido.size(); col++) {
                            filaDeDesconocidos.add(new Integer(PercepcionSnake.DESCONOCIDO));
                        }
                        this.mundoConocido.removeElementAt(0);
                        this.mundoConocido.add(filaDeDesconocidos);

                        // Actualizo la posición del agente
                        Iterator<Point> it = this.composicionDelAgente.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() - 1, par.getY());
                        }

                        // Actualizo la posición de las celdas visitadas
                        it = this.celdasVisitadas.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() - 1, par.getY());
                        }

                        for (int col = 0; col < this.mundoConocido.size(); col++) {
                            hayFilaDeDesconocidos = (this.mundoConocido.firstElement().get(col) == PercepcionSnake.DESCONOCIDO);

                            if (hayFilaDeDesconocidos == false) {
                                break;
                            }
                        }
                    }
                }
            }

            if (!this.encontreElLimiteAlSur) {
                while (hayFilaDeDesconocidos) {
                    Vector<Integer> filaDeDesconocidos = new Vector<Integer>();

                    for (int col = 0; col < this.mundoConocido.size(); col++) {
                        filaDeDesconocidos.add(new Integer(PercepcionSnake.DESCONOCIDO));
                    }
                    this.mundoConocido.removeElementAt(0);
                    this.mundoConocido.add(filaDeDesconocidos);

                    // Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY());
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY());
                    }

                    for (int col = 0; col < this.mundoConocido.size(); col++) {
                        hayFilaDeDesconocidos = (this.mundoConocido.firstElement().get(col) == PercepcionSnake.DESCONOCIDO);

                        if (hayFilaDeDesconocidos == false) {
                            break;
                        }
                    }
                }
            } else if (hayFilaDeDesconocidos && encontreElLimiteAlSur) {
                boolean columnaDeDesconocidosAlOeste = false;
                for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                    columnaDeDesconocidosAlOeste = (this.mundoConocido.get(fil).firstElement() == PercepcionSnake.DESCONOCIDO);

                    if (!columnaDeDesconocidosAlOeste) {
                        break;
                    }
                }

                boolean columnaDeDesconocidosAlEste = false;
                for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                    columnaDeDesconocidosAlEste = (this.mundoConocido.get(fil).lastElement() == PercepcionSnake.DESCONOCIDO);

                    if (!columnaDeDesconocidosAlEste) {
                        break;
                    }
                }

                this.mundoConocido.removeElementAt(0);

                if (columnaDeDesconocidosAlOeste) {
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        this.mundoConocido.get(fil).removeElementAt(0);
//					Actualizo la posición del agente
                    }
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }
                } else if (columnaDeDesconocidosAlEste) {
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        this.mundoConocido.get(fil).removeElementAt(this.mundoConocido.size() - 1);//					Actualizo la posición del agente
                    }
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY());
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY());
                    }
                } else {
//					Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY());
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY());
                    }
                }
            }
        } else {
            if ((int) cabeza.getX() == 0) {
                if (this.mundoConocido.size() == 10 &&
                        !this.encontreElLimiteAlSur &&
                        !this.encontreElLimiteAlNorte) {
                    /*
                     * Si estoy en el límite máximo del mundo, la idea es que seguro habrá algo desconocido en el extremo opuesto,
                     * por lo tanto, determino que esa fila desconocida, estará al norte ahora.
                     * 
                     */
                    Vector<Integer> filaNueva = new Vector<Integer>();

                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        filaNueva.add(new Integer(PercepcionSnake.DESCONOCIDO));
                    }
                    this.mundoConocido.removeElementAt(this.mundoConocido.size() - 1);

                    this.mundoConocido.insertElementAt(filaNueva, 0);

//					Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() + 1, par.getY());
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() + 1, par.getY());
                    }
                } else {
                    // Agrando el mundo conocido
                    // Creo la fila nueva
                    Vector<Integer> filaNueva = new Vector<Integer>();
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        filaNueva.add(new Integer(PercepcionSnake.DESCONOCIDO));                    // Agrego la fila nueva a la matriz del mundo conocido al norte
                    }
                    this.mundoConocido.insertElementAt(filaNueva, 0);

                    // Por defecto, para que la matriz quede cuadrada, agregamos una columna al este.
                    // Si no se puede xq ya se encontró el límite, se agrega al oeste.
                    if (!this.encontreElLimiteAlEste) {
                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            this.mundoConocido.get(fil).add(new Integer(EstadoSnake.DESCONOCIDO));                        // Actualizo la posición del agente
                        }
                        Iterator<Point> it = this.composicionDelAgente.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() + 1, par.getY());
                        }

                        // Actualizo la posición de las celdas visitadas
                        it = this.celdasVisitadas.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() + 1, par.getY());
                        }
                    } else {
                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            this.mundoConocido.get(fil).insertElementAt(new Integer(EstadoSnake.DESCONOCIDO), 0);                        // Actualizo la posición del agente
                        }
                        Iterator<Point> it = this.composicionDelAgente.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() + 1, par.getY() + 1);
                        }

                        // Actualizo la posición de las celdas visitadas
                        it = this.celdasVisitadas.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() + 1, par.getY() + 1);
                        }
                    }
                }
            }
            cabeza = this.composicionDelAgente.firstElement();
            // Agrego la percepción
            this.mundoConocido.get((int) cabeza.getX() - 1).setElementAt(
                    new Integer(((PercepcionSnake) p).getSensorNorte()), (int) cabeza.getY());
        }
        // Al Sur de la cabeza
        if (((PercepcionSnake) p).getSensorSur() == PercepcionSnake.LIMITE_DEL_MUNDO) {
            this.encontreElLimiteAlSur = true;

            // Vemos si hay fila de desconocidos al sur
            boolean hayFilaDeDesconocidos = false;

            for (int col = 0; col < this.mundoConocido.size(); col++) {
                hayFilaDeDesconocidos = (this.mundoConocido.lastElement().get(col) == PercepcionSnake.DESCONOCIDO);

                if (hayFilaDeDesconocidos == false) {
                    break;
                }
            }

            if (this.mundoConocido.size() == 10) {
//				if (!hayFilaDeDesconocidos) {
//					return;
//				}
//				else {
                if (hayFilaDeDesconocidos) {
                    boolean columnaDeDesconocidosAlOeste = false;
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        columnaDeDesconocidosAlOeste = (this.mundoConocido.get(fil).firstElement() == PercepcionSnake.DESCONOCIDO);

                        if (!columnaDeDesconocidosAlOeste) {
                            break;
                        }
                    }

                    boolean columnaDeDesconocidosAlEste = false;
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        columnaDeDesconocidosAlEste = (this.mundoConocido.get(fil).lastElement() == PercepcionSnake.DESCONOCIDO);

                        if (!columnaDeDesconocidosAlEste) {
                            break;
                        }
                    }

                    this.mundoConocido.removeElementAt(this.mundoConocido.size() - 1);

                    while (hayFilaDeDesconocidos) {
                        if (columnaDeDesconocidosAlOeste && hayFilaDeDesconocidos) {
                            for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                                this.mundoConocido.get(fil).removeElementAt(0);
                            //						Actualizo la posición del agente
                            }
                            Iterator<Point> it = this.composicionDelAgente.iterator();
                            while (it.hasNext()) {
                                Point par = it.next();
                                par.setLocation(par.getX(), par.getY() - 1);
                            }

                            // Actualizo la posición de las celdas visitadas
                            it = this.celdasVisitadas.iterator();
                            while (it.hasNext()) {
                                Point par = it.next();
                                par.setLocation(par.getX(), par.getY() - 1);
                            }
                        }
                        if (columnaDeDesconocidosAlEste && hayFilaDeDesconocidos) {
                            for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                                this.mundoConocido.get(fil).removeElementAt(this.mundoConocido.size() - 1);
                            }
                            for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                                columnaDeDesconocidosAlEste = (this.mundoConocido.get(fil).lastElement() == PercepcionSnake.DESCONOCIDO);

                                if (!columnaDeDesconocidosAlEste) {
                                    break;
                                }
                            }
                        }


                        Vector<Integer> filaDeDesconocidos = new Vector<Integer>();

                        for (int col = 0; col < this.mundoConocido.size(); col++) {
                            filaDeDesconocidos.add(new Integer(PercepcionSnake.DESCONOCIDO));
                        }
                        this.mundoConocido.removeElementAt(this.mundoConocido.size() - 1);
                        this.mundoConocido.insertElementAt(filaDeDesconocidos, 0);

                        // Actualizo la posición del agente
                        Iterator<Point> it = this.composicionDelAgente.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() + 1, par.getY());
                        }

                        // Actualizo la posición de las celdas visitadas
                        it = this.celdasVisitadas.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() + 1, par.getY());
                        }

                        for (int col = 0; col < this.mundoConocido.size(); col++) {
                            hayFilaDeDesconocidos = (this.mundoConocido.lastElement().get(col) == PercepcionSnake.DESCONOCIDO);

                            if (hayFilaDeDesconocidos == false) {
                                break;
                            }
                        }
                    }
                }
            }

            if (!this.encontreElLimiteAlNorte) {
                while (hayFilaDeDesconocidos) {
                    Vector<Integer> filaDeDesconocidos = new Vector<Integer>();

                    for (int col = 0; col < this.mundoConocido.size(); col++) {
                        filaDeDesconocidos.add(new Integer(PercepcionSnake.DESCONOCIDO));
                    }
                    this.mundoConocido.removeElementAt(this.mundoConocido.size() - 1);
                    this.mundoConocido.insertElementAt(filaDeDesconocidos, 0);

                    // Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() + 1, par.getY());
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() + 1, par.getY());
                    }

                    for (int col = 0; col < this.mundoConocido.size(); col++) {
                        hayFilaDeDesconocidos = (this.mundoConocido.lastElement().get(col) == PercepcionSnake.DESCONOCIDO);

                        if (hayFilaDeDesconocidos == false) {
                            break;
                        }
                    }
                }
            } else if (hayFilaDeDesconocidos && encontreElLimiteAlNorte) {
                boolean columnaDeDesconocidosAlOeste = false;
                for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                    columnaDeDesconocidosAlOeste = (this.mundoConocido.get(fil).firstElement() == PercepcionSnake.DESCONOCIDO);

                    if (!columnaDeDesconocidosAlOeste) {
                        break;
                    }
                }

                boolean columnaDeDesconocidosAlEste = false;
                for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                    columnaDeDesconocidosAlEste = (this.mundoConocido.get(fil).lastElement() == PercepcionSnake.DESCONOCIDO);

                    if (!columnaDeDesconocidosAlEste) {
                        break;
                    }
                }

                this.mundoConocido.removeElementAt(this.mundoConocido.size() - 1);

                if (columnaDeDesconocidosAlOeste) {
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        this.mundoConocido.get(fil).removeElementAt(0);
//					Actualizo la posición del agente
                    }
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }
                } else if (columnaDeDesconocidosAlEste) {
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        this.mundoConocido.get(fil).removeElementAt(this.mundoConocido.size() - 1);
                    }
                }
            }
        } else {
            if ((int) cabeza.getX() == this.mundoConocido.size() - 1) {
                if (this.mundoConocido.size() == 10 &&
                        !this.encontreElLimiteAlNorte &&
                        !this.encontreElLimiteAlSur) {
                    /*
                     * Si estoy en el límite máximo del mundo, la idea es que seguro habrá algo desconocido en el extremo opuesto,
                     * por lo tanto, determino que esa fila desconocida, estará al sur ahora.
                     * 
                     */
                    Vector<Integer> filaNueva = new Vector<Integer>();

                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        filaNueva.add(new Integer(PercepcionSnake.DESCONOCIDO));
                    }
                    this.mundoConocido.removeElementAt(0);

                    this.mundoConocido.add(filaNueva);

//					Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY());
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY());
                    }
                } else {
                    // Agrando el mundo conocido
                    // Creo la fila nueva
                    Vector<Integer> filaNueva = new Vector<Integer>();
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        filaNueva.add(new Integer(PercepcionSnake.DESCONOCIDO));                    // Agrego la fila nueva a la matriz del mundo conocido al sur
                    }
                    this.mundoConocido.add(filaNueva);

                    // Por defecto, para que la matriz quede cuadrada, agregamos una columna al este.
                    // Si no se puede xq ya se encontró el límite, se agrega al oeste.
                    if (!this.encontreElLimiteAlEste) {
                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            this.mundoConocido.get(fil).add(new Integer(EstadoSnake.DESCONOCIDO));                        // No tengo que actualizar la posición del agente ni las celdas visitadas.
                        }
                    } else {
                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            this.mundoConocido.get(fil).insertElementAt(new Integer(EstadoSnake.DESCONOCIDO), 0);                        // Actualizo la posición del agente
                        }
                        Iterator<Point> it = this.composicionDelAgente.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX(), par.getY() + 1);
                        }

                        // Actualizo la posición de las celdas visitadas
                        it = this.celdasVisitadas.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX(), par.getY() + 1);
                        }
                    }
                }
            }
            cabeza = this.composicionDelAgente.firstElement();
            this.mundoConocido.get((int) cabeza.getX() + 1).setElementAt(
                    new Integer(((PercepcionSnake) p).getSensorSur()), (int) cabeza.getY());
        }
        // Al Este de la cabeza
        if (((PercepcionSnake) p).getSensorEste() == PercepcionSnake.LIMITE_DEL_MUNDO) {
            this.encontreElLimiteAlEste = true;

            // Vemos si hay columna de desconocidos al este
            boolean hayColumnaDeDesconocidos = false;

            for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                hayColumnaDeDesconocidos = (this.mundoConocido.get(fil).lastElement() == PercepcionSnake.DESCONOCIDO);

                if (hayColumnaDeDesconocidos == false) {
                    break;
                }
            }

            if (this.mundoConocido.size() == 10) {
//				if (!hayColumnaDeDesconocidos) 
//					return;
//				else {
                if (hayColumnaDeDesconocidos) {
                    boolean filaDeDesconocidosAlNorte = false;
                    for (int col = 0; col < this.mundoConocido.size(); col++) {
                        filaDeDesconocidosAlNorte = (this.mundoConocido.firstElement().get(col) == PercepcionSnake.DESCONOCIDO);

                        if (!filaDeDesconocidosAlNorte) {
                            break;
                        }
                    }

                    boolean filaDeDesconocidosAlSur = false;
                    for (int col = 0; col < this.mundoConocido.size(); col++) {
                        filaDeDesconocidosAlSur = (this.mundoConocido.lastElement().get(col) == PercepcionSnake.DESCONOCIDO);

                        if (!filaDeDesconocidosAlSur) {
                            break;
                        }
                    }

                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        this.mundoConocido.get(fil).removeElementAt(this.mundoConocido.size() - 1);
                    }
                    while (hayColumnaDeDesconocidos) {
                        if (filaDeDesconocidosAlNorte && hayColumnaDeDesconocidos) {
                            this.mundoConocido.removeElementAt(0);
                            //						Actualizo la posición del agente
                            Iterator<Point> it = this.composicionDelAgente.iterator();
                            while (it.hasNext()) {
                                Point par = it.next();
                                par.setLocation(par.getX() - 1, par.getY());
                            }

                            // Actualizo la posición de las celdas visitadas
                            it = this.celdasVisitadas.iterator();
                            while (it.hasNext()) {
                                Point par = it.next();
                                par.setLocation(par.getX() - 1, par.getY());
                            }

                            for (int col = 0; col < this.mundoConocido.size(); col++) {
                                filaDeDesconocidosAlNorte = (this.mundoConocido.firstElement().get(col) == PercepcionSnake.DESCONOCIDO);

                                if (!filaDeDesconocidosAlNorte) {
                                    break;
                                }
                            }
                        }
                        if (filaDeDesconocidosAlSur && hayColumnaDeDesconocidos) {
                            this.mundoConocido.removeElementAt(this.mundoConocido.size() - 1);

                            for (int col = 0; col < this.mundoConocido.size(); col++) {
                                filaDeDesconocidosAlSur = (this.mundoConocido.lastElement().get(col) == PercepcionSnake.DESCONOCIDO);

                                if (!filaDeDesconocidosAlSur) {
                                    break;
                                }
                            }
                        }


                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            this.mundoConocido.get(fil).removeElementAt(this.mundoConocido.size() - 1);
                            this.mundoConocido.get(fil).insertElementAt(new Integer(PercepcionSnake.DESCONOCIDO), 0);
                        }

                        // Actualizo la posición del agente
                        Iterator<Point> it = this.composicionDelAgente.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX(), par.getY() + 1);
                        }

                        // Actualizo la posición de las celdas visitadas
                        it = this.celdasVisitadas.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX(), par.getY() + 1);
                        }

                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            hayColumnaDeDesconocidos = (this.mundoConocido.get(fil).lastElement() == PercepcionSnake.DESCONOCIDO);

                            if (hayColumnaDeDesconocidos == false) {
                                break;
                            }
                        }
                    }
                }
            }

            if (!this.encontreElLimiteAlOeste) {
                while (hayColumnaDeDesconocidos) {
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        this.mundoConocido.get(fil).removeElementAt(this.mundoConocido.size() - 1);
                        this.mundoConocido.get(fil).insertElementAt(new Integer(PercepcionSnake.DESCONOCIDO), 0);
                    }

                    // Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() + 1);
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() + 1);
                    }

                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        hayColumnaDeDesconocidos = (this.mundoConocido.get(fil).lastElement() == PercepcionSnake.DESCONOCIDO);

                        if (hayColumnaDeDesconocidos == false) {
                            break;
                        }
                    }
                }
            } else if (hayColumnaDeDesconocidos && encontreElLimiteAlOeste) {
                boolean filaDeDesconocidosAlNorte = false;
                for (int col = 0; col < this.mundoConocido.size(); col++) {
                    filaDeDesconocidosAlNorte = (this.mundoConocido.firstElement().get(col) == PercepcionSnake.DESCONOCIDO);

                    if (!filaDeDesconocidosAlNorte) {
                        break;
                    }
                }

                boolean filaDeDesconocidosAlSur = false;
                for (int col = 0; col < this.mundoConocido.size(); col++) {
                    filaDeDesconocidosAlSur = (this.mundoConocido.lastElement().get(col) == PercepcionSnake.DESCONOCIDO);

                    if (!filaDeDesconocidosAlSur) {
                        break;
                    }
                }

                for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                    this.mundoConocido.get(fil).removeElementAt(this.mundoConocido.size() - 1);
                }
                if (filaDeDesconocidosAlNorte) {
                    this.mundoConocido.removeElementAt(0);
//					Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY());
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY());
                    }
                } else if (filaDeDesconocidosAlSur) {
                    this.mundoConocido.removeElementAt(this.mundoConocido.size() - 1);
                }

            }
        } else {
            if ((int) cabeza.getY() == this.mundoConocido.size() - 1) {
                if (this.mundoConocido.size() == 10 &&
                        !this.encontreElLimiteAlOeste &&
                        !this.encontreElLimiteAlEste) {
                    /*
                     * Si estoy en el límite máximo del mundo, la idea es que seguro habrá algo desconocido en el extremo opuesto,
                     * por lo tanto, determino que esa columna desconocida, estará al este ahora.
                     * 
                     */

                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        this.mundoConocido.get(fil).removeElementAt(0);
                        this.mundoConocido.get(fil).add(new Integer(PercepcionSnake.DESCONOCIDO));
                    }

//					Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }
                } else {
                    // Agrando el mundo conocido
                    // Creo la fila nueva
                    Vector<Integer> filaNueva = new Vector<Integer>();
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        filaNueva.add(new Integer(PercepcionSnake.DESCONOCIDO));                    // Por defecto, para que la matriz quede cuadrada, agregamos una fila al sur.
                    // Si no se puede xq ya se encontró el límite, se agrega al norte.
                    // Agrego la fila nueva a la matriz del mundo conocido al sur
                    }
                    if (!this.encontreElLimiteAlSur) {
                        this.mundoConocido.add(filaNueva);

                        // Agregamos las columnas
                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            this.mundoConocido.get(fil).add(new Integer(EstadoSnake.DESCONOCIDO));
                        }
                    } else {
                        this.mundoConocido.insertElementAt(filaNueva, 0);

                        // Agregamos las columnas
                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            this.mundoConocido.get(fil).add(new Integer(EstadoSnake.DESCONOCIDO));                        // Actualizo la posición del agente
                        }
                        Iterator<Point> it = this.composicionDelAgente.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() + 1, par.getY());
                        }

                        // Actualizo la posición de las celdas visitadas
                        it = this.celdasVisitadas.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() + 1, par.getY());
                        }
                    }
                }
            }

            cabeza = this.composicionDelAgente.firstElement();
            this.mundoConocido.get((int) cabeza.getX()).setElementAt(
                    new Integer(((PercepcionSnake) p).getSensorEste()), (int) cabeza.getY() + 1);
        }
        // Al Oeste de la cabeza
        if (((PercepcionSnake) p).getSensorOeste() == PercepcionSnake.LIMITE_DEL_MUNDO) {
            this.encontreElLimiteAlOeste = true;

            // Vemos si hay columna de desconocidos al oeste
            boolean hayColumnaDeDesconocidos = false;

            for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                hayColumnaDeDesconocidos = (this.mundoConocido.get(fil).firstElement() == PercepcionSnake.DESCONOCIDO);

                if (hayColumnaDeDesconocidos == false) {
                    break;
                }
            }

            if (this.mundoConocido.size() == 10) {
//				if (!hayColumnaDeDesconocidos)
//					return;
//				else {
                if (hayColumnaDeDesconocidos) {
                    boolean filaDeDesconocidosAlNorte = false;
                    for (int col = 0; col < this.mundoConocido.size(); col++) {
                        filaDeDesconocidosAlNorte = (this.mundoConocido.firstElement().get(col) == PercepcionSnake.DESCONOCIDO);

                        if (!filaDeDesconocidosAlNorte) {
                            break;
                        }
                    }

                    boolean filaDeDesconocidosAlSur = false;
                    for (int col = 0; col < this.mundoConocido.size(); col++) {
                        filaDeDesconocidosAlSur = (this.mundoConocido.lastElement().get(col) == PercepcionSnake.DESCONOCIDO);

                        if (!filaDeDesconocidosAlSur) {
                            break;
                        }
                    }

                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        this.mundoConocido.get(fil).removeElementAt(0);
                    }
                    while (hayColumnaDeDesconocidos) {
                        if (filaDeDesconocidosAlNorte && hayColumnaDeDesconocidos) {
                            this.mundoConocido.removeElementAt(0);
                            //						Actualizo la posición del agente
                            Iterator<Point> it = this.composicionDelAgente.iterator();
                            while (it.hasNext()) {
                                Point par = it.next();
                                par.setLocation(par.getX() - 1, par.getY());
                            }

                            // Actualizo la posición de las celdas visitadas
                            it = this.celdasVisitadas.iterator();
                            while (it.hasNext()) {
                                Point par = it.next();
                                par.setLocation(par.getX() - 1, par.getY());
                            }

                            for (int col = 0; col < this.mundoConocido.size(); col++) {
                                filaDeDesconocidosAlNorte = (this.mundoConocido.firstElement().get(col) == PercepcionSnake.DESCONOCIDO);

                                if (!filaDeDesconocidosAlNorte) {
                                    break;
                                }
                            }

                        }
                        if (filaDeDesconocidosAlSur && hayColumnaDeDesconocidos) {
                            this.mundoConocido.removeElementAt(this.mundoConocido.size() - 1);

                            for (int col = 0; col < this.mundoConocido.size(); col++) {
                                filaDeDesconocidosAlSur = (this.mundoConocido.lastElement().get(col) == PercepcionSnake.DESCONOCIDO);

                                if (!filaDeDesconocidosAlSur) {
                                    break;
                                }
                            }
                        }


                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            this.mundoConocido.get(fil).removeElementAt(0);
                            this.mundoConocido.get(fil).add(new Integer(PercepcionSnake.DESCONOCIDO));
                        }

                        // Actualizo la posición del agente
                        Iterator<Point> it = this.composicionDelAgente.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX(), par.getY() - 1);
                        }

                        // Actualizo la posición de las celdas visitadas
                        it = this.celdasVisitadas.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX(), par.getY() - 1);
                        }

                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            hayColumnaDeDesconocidos = (this.mundoConocido.get(fil).firstElement() == PercepcionSnake.DESCONOCIDO);

                            if (hayColumnaDeDesconocidos == false) {
                                break;
                            }
                        }
                    }

//					// Actualizo la posición del agente
//					Iterator<Point> it =  this.composicionDelAgente.iterator();
//					while (it.hasNext()) {
//						Point par = it.next();
//						par.setLocation(par.getX(), par.getY() - 1);
//					}
//
//					// Actualizo la posición de las celdas visitadas
//					it =  this.celdasVisitadas.iterator();
//					while (it.hasNext()) {
//						Point par = it.next();
//						par.setLocation(par.getX(), par.getY() - 1);
//					}

                }
            }

            if (!this.encontreElLimiteAlEste) {
                while (hayColumnaDeDesconocidos) {
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        this.mundoConocido.get(fil).removeElementAt(0);
                        this.mundoConocido.get(fil).add(new Integer(PercepcionSnake.DESCONOCIDO));
                    }

                    // Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }

                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        hayColumnaDeDesconocidos = (this.mundoConocido.get(fil).firstElement() == PercepcionSnake.DESCONOCIDO);

                        if (hayColumnaDeDesconocidos == false) {
                            break;
                        }
                    }
                }
            } else if (hayColumnaDeDesconocidos && encontreElLimiteAlEste) {
                boolean filaDeDesconocidosAlNorte = false;
                for (int col = 0; col < this.mundoConocido.size(); col++) {
                    filaDeDesconocidosAlNorte = (this.mundoConocido.firstElement().get(col) == PercepcionSnake.DESCONOCIDO);

                    if (!filaDeDesconocidosAlNorte) {
                        break;
                    }
                }

                boolean filaDeDesconocidosAlSur = false;
                for (int col = 0; col < this.mundoConocido.size(); col++) {
                    filaDeDesconocidosAlSur = (this.mundoConocido.lastElement().get(col) == PercepcionSnake.DESCONOCIDO);

                    if (!filaDeDesconocidosAlSur) {
                        break;
                    }
                }

                for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                    this.mundoConocido.get(fil).removeElementAt(0);
                }
                if (filaDeDesconocidosAlNorte) {
                    this.mundoConocido.removeElementAt(0);
//					Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY() - 1);
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX() - 1, par.getY() - 1);
                    }
                } else if (filaDeDesconocidosAlSur) {
                    this.mundoConocido.removeElementAt(this.mundoConocido.size() - 1);

                    // Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }
                } else {
//					Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() - 1);
                    }
                }
            }
        } else {
            if ((int) cabeza.getY() == 0) {
                if (this.mundoConocido.size() == 10 &&
                        !this.encontreElLimiteAlEste &&
                        !this.encontreElLimiteAlOeste) {
                    /*
                     * Si estoy en el límite máximo del mundo, la idea es que seguro habrá algo desconocido en el extremo opuesto,
                     * por lo tanto, determino que esa columna desconocida, estará al oeste ahora.
                     * 
                     */

                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        this.mundoConocido.get(fil).removeElementAt(this.mundoConocido.size() - 1);
                        this.mundoConocido.get(fil).insertElementAt(new Integer(PercepcionSnake.DESCONOCIDO), 0);
                    }

//					Actualizo la posición del agente
                    Iterator<Point> it = this.composicionDelAgente.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() + 1);
                    }

                    // Actualizo la posición de las celdas visitadas
                    it = this.celdasVisitadas.iterator();
                    while (it.hasNext()) {
                        Point par = it.next();
                        par.setLocation(par.getX(), par.getY() + 1);
                    }
                } else {
                    // Agrando el mundo conocido

                    // Creo la fila nueva
                    Vector<Integer> filaNueva = new Vector<Integer>();
                    for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                        filaNueva.add(new Integer(PercepcionSnake.DESCONOCIDO));                    // Por defecto, para que la matriz quede cuadrada, agregamos una fila al sur.
                    // Si no se puede xq ya se encontró el límite, se agrega al norte.
                    // Agrego la fila nueva a la matriz del mundo conocido al sur
                    }
                    if (!this.encontreElLimiteAlSur) {
                        this.mundoConocido.add(filaNueva);

                        // Agregamos las columnas
                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            this.mundoConocido.get(fil).insertElementAt(new Integer(EstadoSnake.DESCONOCIDO), 0);                        // Actualizo la posición del agente
                        }
                        Iterator<Point> it = this.composicionDelAgente.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX(), par.getY() + 1);
                        }

                        // Actualizo la posición de las celdas visitadas
                        it = this.celdasVisitadas.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX(), par.getY() + 1);
                        }
                    } else {
                        this.mundoConocido.insertElementAt(filaNueva, 0);

                        // Agregamos las columnas
                        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
                            this.mundoConocido.get(fil).insertElementAt(new Integer(EstadoSnake.DESCONOCIDO), 0);                        // Actualizo la posición del agente
                        }
                        Iterator<Point> it = this.composicionDelAgente.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() + 1, par.getY() + 1);
                        }

                        // Actualizo la posición de las celdas visitadas
                        it = this.celdasVisitadas.iterator();
                        while (it.hasNext()) {
                            Point par = it.next();
                            par.setLocation(par.getX() + 1, par.getY() + 1);
                        }
                    }
                }
            }
            cabeza = this.composicionDelAgente.firstElement();
            this.mundoConocido.get((int) cabeza.getX()).setElementAt(
                    new Integer(((PercepcionSnake) p).getSensorOeste()), (int) cabeza.getY() - 1);
        }
    }

    @Override
    public SearchBasedAgentState clone() {
        EstadoSnake clon = new EstadoSnake();

        try {
            clon.setLongitud(this.longitud);
        } catch (Exception e) {
        }

        clon.setOrientacionDeLaCabeza(this.orientacionDeLaCabeza);

        Vector<Point> composicionDelAgenteClon = new Vector<Point>();
        for (int par = 0; par < this.composicionDelAgente.size(); par++) {
            composicionDelAgenteClon.add(new Point(
                    this.composicionDelAgente.get(par)));
        }

        clon.setComposicionDelAgente(composicionDelAgenteClon);

        Vector<Point> celdasVisitadasClon = new Vector<Point>();
        for (int par = 0; par < this.celdasVisitadas.size(); par++) {
            celdasVisitadasClon.add(new Point(
                    this.celdasVisitadas.get(par)));
        }

        clon.setCeldasVisitadas(celdasVisitadasClon);

        Vector<Vector<Integer>> mundoConocidoClon = new Vector<Vector<Integer>>();
        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
            mundoConocidoClon.add(new Vector<Integer>());
            for (int col = 0; col < this.mundoConocido.get(fil).size(); col++) {
                mundoConocidoClon.get(fil).add(new Integer(
                        this.mundoConocido.get(fil).get(col)));
            }
        }

        clon.setMundoConocido(mundoConocidoClon);

        clon.setEstaVivo(this.estaVivo);

        clon.setEncontreElLimiteAlEste(encontreElLimiteAlEste);
        clon.setEncontreElLimiteAlNorte(encontreElLimiteAlNorte);
        clon.setEncontreElLimiteAlOeste(encontreElLimiteAlOeste);
        clon.setEncontreElLimiteAlSur(encontreElLimiteAlSur);

        clon.setVecesQueSeGiro(this.vecesQueSeGiro);
        clon.setVecesQueSeGiroDerecha(this.vecesQueSeGiroDerecha);

        return clon;
    }

    @Override
    public boolean equals(Object obj) {
        EstadoSnake snake = (EstadoSnake) obj;

        // Comparamos si esta vivo
        if (this.estaVivo != snake.getVivo()) {
            return false;        // Comparamos los limites encontrados
        }
        if (this.encontreElLimiteAlEste != snake.isEncontreElLimiteAlEste()) {
            return false;
        }
        if (this.encontreElLimiteAlOeste != snake.isEncontreElLimiteAlOeste()) {
            return false;
        }
        if (this.encontreElLimiteAlNorte != snake.isEncontreElLimiteAlNorte()) {
            return false;
        }
        if (this.encontreElLimiteAlSur != snake.isEncontreElLimiteAlSur()) {
            return false;        // Comparamos la orientacion
        }
        if (this.orientacionDeLaCabeza != snake.getOrientacionDeLaCabeza()) {
            return false;        // Comparamos la longitud
        }
        if (this.longitud != snake.getLongitud()) {
            return false;        // Comparamos la composición del agente
        }
        for (int i = 0; i < this.composicionDelAgente.size(); i++) {
            if (this.composicionDelAgente.get(i) != snake.composicionDelAgente.get(i)) {
                return false;            // Comparamos las celdas visitadas
            }
        }
        if (this.celdasVisitadas.size() != snake.getCeldasVisitadas().size()) {
            return false;
        }
        for (int i = 0; i < this.celdasVisitadas.size(); i++) {
            if (this.celdasVisitadas.get(i) != snake.getCeldasVisitadas().get(i)) {
                return false;            // Comparamos las mundos conocidos
            }
        }
        if (this.mundoConocido.size() != snake.getMundoConocido().size()) {
            return false;
        }
        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
            if (this.mundoConocido.get(fil).size() != snake.getMundoConocido().get(fil).size()) {
                return false;
            }
            for (int col = 0; col < this.mundoConocido.size(); col++) {
                if (this.mundoConocido.get(fil).get(col) != snake.getMundoConocido().get(fil).get(col)) {
                    return false;
                }
            }
        }

        return true;
    }

    @Override
    public void initState() {
        this.mundoConocido = new Vector<Vector<Integer>>();
        // Inicialmente asumimos que el mundo es de 3x3 y desconocido menos la posición central, en donde está el agente.

        for (int fil = 0; fil < 3; fil++) {
            this.mundoConocido.add(new Vector<Integer>());
            for (int col = 0; col < 3; col++) {
                this.mundoConocido.get(fil).add(new Integer(PercepcionSnake.DESCONOCIDO));
            }
        }

        this.mundoConocido.get(1).setElementAt(new Integer(PercepcionSnake.PARTE_DEL_AGENTE), 1);

        // Por defecto el agente arranca mirando al Este.
        this.orientacionDeLaCabeza = EstadoSnake.ESTE;

        this.composicionDelAgente = new Vector<Point>();
        this.composicionDelAgente.add(new Point(1, 1));

        this.celdasVisitadas = new Vector<Point>();

        this.longitud = 1;

        this.estaVivo = true;

        this.encontreElLimiteAlNorte = false;
        this.encontreElLimiteAlSur = false;
        this.encontreElLimiteAlEste = false;
        this.encontreElLimiteAlOeste = false;

        this.vecesQueSeGiro = 0;
        this.vecesQueSeGiroDerecha = 0;
    }

    @Override
    public String toString() {
        String str = "";

        str = str + " composicion=\"{";
        for (int i = 0; i < this.composicionDelAgente.size(); i++) {
            str = str + "(" + (int) this.composicionDelAgente.get(i).getX() + "," + (int) this.composicionDelAgente.get(i).getY() + ")";
        }
        str = str + "}\"\n";
        str = str + " longitud=\"" + this.longitud + "\"\n";
        str = str + " orientacion_cabeza=\"" + this.orientacionComoString() + "\"\n";
        str = str + " esta_vivo=\"" + this.estaVivo + "\"\n";

        str = str + "mundo_conocido=\"[ \n";
        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
            str = str + "[ ";
            for (int col = 0; col < this.mundoConocido.get(fil).size(); col++) {
                if (this.mundoConocido.get(fil).get(col) == -1) {
                    str = str + "* ";
                } else {
                    str = str + this.mundoConocido.get(fil).get(col) + " ";
                }
            }
            str = str + " ]\n";
        }
        str = str + " ]\"";

        return str;
    }

    private String orientacionComoString() {
        String str = "";

        switch (this.orientacionDeLaCabeza) {
            case 0:
                str = "Norte";
                break;
            case 1:
                str = "Sur";
                break;
            case 2:
                str = "Este";
                break;
            case 3:
                str = "Oeste";
                break;
            default:
                break;
        }

        return str;
    }

    public boolean getVivo() {
        return this.estaVivo;
    }

    public int getLongitud() {
        return this.longitud;
    }

    public void setLongitud(int longitud) throws Exception {
        if (longitud != this.composicionDelAgente.size()) {
            throw new Exception("La longitud no concuerda con el tama;o de la vibora");
        }
        this.longitud = longitud;
    }

    public boolean noHayMasComida() {
        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
            for (int col = 0; col < this.mundoConocido.get(fil).size(); col++) {
                if (this.mundoConocido.get(fil).get(col) == EstadoSnake.COMIDA) {
                    return false;
                }
            }
        }
        return true;
    }

    public boolean todoConocido() {
        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
            for (int col = 0; col < this.mundoConocido.get(fil).size(); col++) {
                if (this.mundoConocido.get(fil).get(col) == EstadoSnake.DESCONOCIDO) {
                    return false;
                }
            }
        }
        return true;
    }

    public int getOrientacionDeLaCabeza() {
        return orientacionDeLaCabeza;
    }

    public void setOrientacionDeLaCabeza(int orientacionDeLaCabeza) {
        this.orientacionDeLaCabeza = orientacionDeLaCabeza;
    }

    public Point getPosicionCabeza() {
        return this.composicionDelAgente.firstElement();
    }

    private void setComposicionDelAgente(Vector<Point> composicionDelAgente) {
        this.composicionDelAgente = composicionDelAgente;
    }

    public void setMundoConocido(Vector<Vector<Integer>> mundoConocido) {
        this.mundoConocido = mundoConocido;
    }

    public Vector<Vector<Integer>> getMundoConocido() {
        return this.mundoConocido;
    }

    public void moverAlNorte() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX() - 1,
                (int) this.composicionDelAgente.firstElement().getY()), 0);

        Point puntoABorrar = this.composicionDelAgente.remove(this.composicionDelAgente.size() - 1);

        this.mundoConocido.get((int) this.composicionDelAgente.firstElement().getX()).setElementAt(
                EstadoSnake.PARTE_DEL_AGENTE, (int) this.composicionDelAgente.firstElement().getY());
        this.mundoConocido.get((int) puntoABorrar.getX()).setElementAt(
                EstadoSnake.NO_HAY_NADA, (int) puntoABorrar.getY());

        this.celdasVisitadas.add(new Point((int) puntoABorrar.getX(), (int) puntoABorrar.getY()));
    }

    public void moverAlSur() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX() + 1,
                (int) this.composicionDelAgente.firstElement().getY()), 0);

        Point puntoABorrar = this.composicionDelAgente.remove(this.composicionDelAgente.size() - 1);

        this.mundoConocido.get((int) this.composicionDelAgente.firstElement().getX()).setElementAt(
                EstadoSnake.PARTE_DEL_AGENTE, (int) this.composicionDelAgente.firstElement().getY());
        this.mundoConocido.get((int) puntoABorrar.getX()).setElementAt(
                EstadoSnake.NO_HAY_NADA, (int) puntoABorrar.getY());

        this.celdasVisitadas.add(new Point((int) puntoABorrar.getX(), (int) puntoABorrar.getY()));
    }

    public void moverAlEste() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX(),
                (int) this.composicionDelAgente.firstElement().getY() + 1), 0);

        Point puntoABorrar = this.composicionDelAgente.remove(this.composicionDelAgente.size() - 1);
        this.mundoConocido.get((int) this.composicionDelAgente.firstElement().getX()).setElementAt(
                EstadoSnake.PARTE_DEL_AGENTE, (int) this.composicionDelAgente.firstElement().getY());
        this.mundoConocido.get((int) puntoABorrar.getX()).setElementAt(
                EstadoSnake.NO_HAY_NADA, (int) puntoABorrar.getY());

        this.celdasVisitadas.add(new Point((int) puntoABorrar.getX(), (int) puntoABorrar.getY()));
    }

    public void moverAlOeste() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX(),
                (int) this.composicionDelAgente.firstElement().getY() - 1), 0);

        Point puntoABorrar = this.composicionDelAgente.remove(this.composicionDelAgente.size() - 1);

        this.mundoConocido.get((int) this.composicionDelAgente.firstElement().getX()).setElementAt(
                EstadoSnake.PARTE_DEL_AGENTE, (int) this.composicionDelAgente.firstElement().getY());
        this.mundoConocido.get((int) puntoABorrar.getX()).setElementAt(
                EstadoSnake.NO_HAY_NADA, (int) puntoABorrar.getY());

        this.celdasVisitadas.add(new Point((int) puntoABorrar.getX(), (int) puntoABorrar.getY()));
    }

    public void inventaLoQueTiene() {
        // Ahora pongo todo lo desconocido como si fuera que no hay nada ahi.
        for (int fil = 0; fil < this.mundoConocido.size(); fil++) {
            for (int col = 0; col < this.mundoConocido.get(fil).size(); col++) {
                if (this.mundoConocido.get(fil).get(col) == EstadoSnake.DESCONOCIDO) {
                    this.mundoConocido.get(fil).setElementAt(EstadoSnake.NO_HAY_NADA, col);
                }
            }
        }
    }

    public void comerAlNorte() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX() - 1,
                (int) this.composicionDelAgente.firstElement().getY()), 0);

        this.mundoConocido.get((int) this.composicionDelAgente.firstElement().getX()).setElementAt(
                EstadoSnake.PARTE_DEL_AGENTE, (int) this.composicionDelAgente.firstElement().getY());

        this.longitud = this.composicionDelAgente.size();
    }

    public void comerAlSur() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX() + 1,
                (int) this.composicionDelAgente.firstElement().getY()), 0);

        this.mundoConocido.get((int) this.composicionDelAgente.firstElement().getX()).setElementAt(
                EstadoSnake.PARTE_DEL_AGENTE, (int) this.composicionDelAgente.firstElement().getY());

        this.longitud = this.composicionDelAgente.size();
    }

    public void comerAlEste() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX(),
                (int) this.composicionDelAgente.firstElement().getY() + 1), 0);

        this.mundoConocido.get((int) this.composicionDelAgente.firstElement().getX()).setElementAt(
                EstadoSnake.PARTE_DEL_AGENTE, (int) this.composicionDelAgente.firstElement().getY());

        this.longitud = this.composicionDelAgente.size();
    }

    public void comerAlOeste() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX(),
                (int) this.composicionDelAgente.firstElement().getY() - 1), 0);

        this.mundoConocido.get((int) this.composicionDelAgente.firstElement().getX()).setElementAt(
                EstadoSnake.PARTE_DEL_AGENTE, (int) this.composicionDelAgente.firstElement().getY());

        this.longitud = this.composicionDelAgente.size();
    }

    public boolean hayObstaculo(int donde) {
        Point posicionCabeza = this.getPosicionCabeza();
        boolean hayObstaculo = false;

        switch (donde) {
            case 0:	// Norte
                if (posicionCabeza.getX() == 0) {
                    hayObstaculo = true;
                } else if (this.mundoConocido.get((int) posicionCabeza.getX() - 1).
                        get((int) posicionCabeza.getY()) == EstadoSnake.PARTE_DEL_AGENTE) {
                    hayObstaculo = true;
                }
                break;
            case 1:	// Sur
                if (posicionCabeza.getX() == this.mundoConocido.size() - 1) {
                    hayObstaculo = true;
                } else if (this.mundoConocido.get((int) posicionCabeza.getX() + 1).
                        get((int) posicionCabeza.getY()) == EstadoSnake.PARTE_DEL_AGENTE) {
                    hayObstaculo = true;
                }
                break;
            case 2:	// Este
                if (posicionCabeza.getY() == this.mundoConocido.size() - 1) {
                    hayObstaculo = true;
                } else if (this.mundoConocido.get((int) posicionCabeza.getX()).
                        get((int) posicionCabeza.getY() + 1) == EstadoSnake.PARTE_DEL_AGENTE) {
                    hayObstaculo = true;
                }
                break;
            case 3:	// Oeste

                if (posicionCabeza.getY() == 0) {
                    hayObstaculo = true;
                } else if (this.mundoConocido.get((int) posicionCabeza.getX()).
                        get((int) posicionCabeza.getY() - 1) == EstadoSnake.PARTE_DEL_AGENTE) {
                    hayObstaculo = true;
                }
                break;
            default:
                break;
        }

        return hayObstaculo;
    }

    public boolean hayComida(int donde) {
        Point posicionCabeza = this.getPosicionCabeza();
        boolean hayComida = false;

        switch (donde) {
            case 0:	// Norte
                if (posicionCabeza.getX() == 0) {
                    hayComida = false;
                } else if (this.mundoConocido.get((int) posicionCabeza.getX() - 1).
                        get((int) posicionCabeza.getY()) == EstadoSnake.COMIDA) {
                    hayComida = true;
                }
                break;
            case 1:	// Sur
                if (posicionCabeza.getX() == this.mundoConocido.size() - 1) {
                    hayComida = false;
                } else if (this.mundoConocido.get((int) posicionCabeza.getX() + 1).
                        get((int) posicionCabeza.getY()) == EstadoSnake.COMIDA) {
                    hayComida = true;
                }
                break;
            case 2:	// Este
                if (posicionCabeza.getY() == this.mundoConocido.size() - 1) {
                    hayComida = false;
                } else if (this.mundoConocido.get((int) posicionCabeza.getX()).
                        get((int) posicionCabeza.getY() + 1) == EstadoSnake.COMIDA) {
                    hayComida = true;
                }
                break;
            case 3:	// Oeste
                if (posicionCabeza.getY() == 0) {
                    hayComida = false;
                } else if (this.mundoConocido.get((int) posicionCabeza.getX()).
                        get((int) posicionCabeza.getY() - 1) == EstadoSnake.COMIDA) {
                    hayComida = true;
                }
                break;
            default:
                break;
        }

        return hayComida;
    }

    public boolean isEstaVivo() {
        return estaVivo;
    }

    public void setEstaVivo(boolean estaVivo) {
        this.estaVivo = estaVivo;
    }

    public double getCosto() {
        return this.costo;
    }

    public void incrementarCosto(double costo) {
        this.costo += costo;
    }

    public Vector<Point> getCeldasVisitadas() {
        return celdasVisitadas;
    }

    public boolean pasoNVecesPorEstaCelda(Point celda) {
        int cantidadDeVeces = 0;

        for (int i = 0; i < this.celdasVisitadas.size(); i++) {
            if (this.celdasVisitadas.get(i).getX() == celda.getX() &&
                    this.celdasVisitadas.get(i).getY() == celda.getY()) {
                cantidadDeVeces++;
            }
        }

        if (cantidadDeVeces > 1) {
            return true;
        }
        return false;
    }

    public void agregarCeldaALasVisitadas(Point celda) {
        if (!this.pasoNVecesPorEstaCelda(celda)) {
            this.celdasVisitadas.add(celda);
        }
    }

    public void setCeldasVisitadas(Vector<Point> celdasVisitadas) {
        this.celdasVisitadas = celdasVisitadas;
    }

    public boolean isEncontreElLimiteAlEste() {
        return encontreElLimiteAlEste;
    }

    public void setEncontreElLimiteAlEste(boolean encontreElLimiteAlEste) {
        this.encontreElLimiteAlEste = encontreElLimiteAlEste;
    }

    public boolean isEncontreElLimiteAlNorte() {
        return encontreElLimiteAlNorte;
    }

    public void setEncontreElLimiteAlNorte(boolean encontreElLimiteAlNorte) {
        this.encontreElLimiteAlNorte = encontreElLimiteAlNorte;
    }

    public boolean isEncontreElLimiteAlOeste() {
        return encontreElLimiteAlOeste;
    }

    public void setEncontreElLimiteAlOeste(boolean encontreElLimiteAlOeste) {
        this.encontreElLimiteAlOeste = encontreElLimiteAlOeste;
    }

    public boolean isEncontreElLimiteAlSur() {
        return encontreElLimiteAlSur;
    }

    public void setEncontreElLimiteAlSur(boolean encontreElLimiteAlSur) {
        this.encontreElLimiteAlSur = encontreElLimiteAlSur;
    }

    public void destrabaAlAgente() {
        this.celdasVisitadas.clear();
    }

    public int getVecesQueSeGiro() {
        return this.vecesQueSeGiro;
    }

    public void setVecesQueSeGiro(int vecesQueSeGiro) {
        this.vecesQueSeGiro = vecesQueSeGiro;
    }

    public void incVecesQueSeGiro() {
        this.vecesQueSeGiro++;
    }

    public int getVecesQueSeGiroDerecha() {
        return this.vecesQueSeGiroDerecha;
    }

    public void setVecesQueSeGiroDerecha(int vecesQueSeGiroDerecha) {
        this.vecesQueSeGiroDerecha = vecesQueSeGiroDerecha;
    }

    public void incVecesQueSeGiroDerecha() {
        this.vecesQueSeGiroDerecha++;
    }
}

## Goal definition

In [36]:
public class MetaSnake extends GoalTest {

    @Override
    public boolean isGoalState(AgentState agentState) {
        if (((EstadoSnake) agentState).noHayMasComida() &&
                ((EstadoSnake) agentState).todoConocido() &&
                ((EstadoSnake) agentState).getVivo()) {
            return true;
        }
        return false;
    }
}

## Actions definition

In [37]:
public class Avanzar extends SearchAction {

    @Override
    public SearchBasedAgentState execute(SearchBasedAgentState s) {
        EstadoSnake snake = (EstadoSnake) s;
        snake.incrementarCosto(this.getCost());

        if (snake.getOrientacionDeLaCabeza() == EstadoSnake.NORTE &&
                !snake.hayObstaculo(EstadoSnake.NORTE) &&
                !snake.hayComida(EstadoSnake.NORTE) &&
                !snake.pasoNVecesPorEstaCelda(new Point(
                (int) snake.getPosicionCabeza().getX() - 1,
                (int) snake.getPosicionCabeza().getY()))) {
            snake.moverAlNorte();

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.SUR &&
                !snake.hayObstaculo(EstadoSnake.SUR) &&
                !snake.hayComida(EstadoSnake.SUR) &&
                !snake.pasoNVecesPorEstaCelda(new Point(
                (int) snake.getPosicionCabeza().getX() + 1,
                (int) snake.getPosicionCabeza().getY()))) {
            snake.moverAlSur();

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.ESTE &&
                !snake.hayObstaculo(EstadoSnake.ESTE) &&
                !snake.hayComida(EstadoSnake.ESTE) &&
                !snake.pasoNVecesPorEstaCelda(new Point(
                (int) snake.getPosicionCabeza().getX(),
                (int) snake.getPosicionCabeza().getY() + 1))) {
            snake.moverAlEste();

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.OESTE &&
                !snake.hayObstaculo(EstadoSnake.OESTE) &&
                !snake.hayComida(EstadoSnake.OESTE) &&
                !snake.pasoNVecesPorEstaCelda(new Point(
                (int) snake.getPosicionCabeza().getX(),
                (int) snake.getPosicionCabeza().getY() - 1))) {
            snake.moverAlOeste();

            return snake;
        }

        return null;
    }

    @Override
    public EnvironmentState execute(AgentState ast, EnvironmentState est) {
        EstadoAmbiente estA = (EstadoAmbiente) est;
        EstadoSnake snake = (EstadoSnake) ast;

        if (snake.getOrientacionDeLaCabeza() == EstadoSnake.NORTE &&
                !snake.hayObstaculo(EstadoSnake.NORTE) &&
                !snake.hayComida(EstadoSnake.NORTE) &&
                !snake.pasoNVecesPorEstaCelda(new Point(
                (int) snake.getPosicionCabeza().getX() - 1,
                (int) snake.getPosicionCabeza().getY()))) {
            snake.moverAlNorte();
            estA.moverAlNorte();
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.SUR &&
                !snake.hayObstaculo(EstadoSnake.SUR) &&
                !snake.hayComida(EstadoSnake.SUR) &&
                !snake.pasoNVecesPorEstaCelda(new Point(
                (int) snake.getPosicionCabeza().getX() + 1,
                (int) snake.getPosicionCabeza().getY()))) {
            snake.moverAlSur();
            estA.moverAlSur();
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.ESTE &&
                !snake.hayObstaculo(EstadoSnake.ESTE) &&
                !snake.hayComida(EstadoSnake.ESTE) &&
                !snake.pasoNVecesPorEstaCelda(new Point(
                (int) snake.getPosicionCabeza().getX(),
                (int) snake.getPosicionCabeza().getY() + 1))) {
            snake.moverAlEste();
            estA.moverAlEste();
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.OESTE &&
                !snake.hayObstaculo(EstadoSnake.OESTE) &&
                !snake.hayComida(EstadoSnake.OESTE) &&
                !snake.pasoNVecesPorEstaCelda(new Point(
                (int) snake.getPosicionCabeza().getX(),
                (int) snake.getPosicionCabeza().getY() - 1))) {
            snake.moverAlOeste();
            estA.moverAlOeste();
        }

        return estA;
    }

    @Override
    public Double getCost() {
        return new Double(7);
    }

    @Override
    public String toString() {
        return "Avanzar";
    }
}

In [38]:
public class Comer extends SearchAction {

    @Override
    public SearchBasedAgentState execute(SearchBasedAgentState s) {
        EstadoSnake snake = (EstadoSnake) s;
        snake.incrementarCosto(this.getCost());

        if (snake.getOrientacionDeLaCabeza() == EstadoSnake.NORTE &&
                snake.hayComida(EstadoSnake.NORTE)) {
            snake.comerAlNorte();

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.SUR &&
                snake.hayComida(EstadoSnake.SUR)) {
            snake.comerAlSur();

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.ESTE &&
                snake.hayComida(EstadoSnake.ESTE)) {
            snake.comerAlEste();

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.OESTE &&
                snake.hayComida(EstadoSnake.OESTE)) {
            snake.comerAlOeste();

            return snake;
        }

        return null;
    }

    @Override
    public EnvironmentState execute(AgentState ast, EnvironmentState est) {
        EstadoAmbiente estA = (EstadoAmbiente) est;
        EstadoSnake snake = (EstadoSnake) ast;

        if (snake.getOrientacionDeLaCabeza() == EstadoSnake.NORTE &&
                snake.hayComida(EstadoSnake.NORTE)) {
            snake.comerAlNorte();
            estA.comerAlNorte();
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.SUR &&
                snake.hayComida(EstadoSnake.SUR)) {
            snake.comerAlSur();
            estA.comerAlSur();
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.ESTE &&
                snake.hayComida(EstadoSnake.ESTE)) {
            snake.comerAlEste();
            estA.comerAlEste();
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.OESTE &&
                snake.hayComida(EstadoSnake.OESTE)) {
            snake.comerAlOeste();
            estA.comerAlOeste();
        }

        return estA;
    }

    @Override
    public Double getCost() {
        return new Double(5);
    }

    @Override
    public String toString() {
        return "Comer";
    }
}

In [39]:
public class GirarDerecha extends SearchAction {

    @Override
    public SearchBasedAgentState execute(SearchBasedAgentState s) {
        EstadoSnake snake = (EstadoSnake) s;
        snake.incrementarCosto(this.getCost());

//		if (snake.getVecesQueSeGiroDerecha() > 4) {
//			snake.destrabaAlAgente();
//			snake.setVecesQueSeGiroDerecha(0);
//			
//			return null;
//		}

        if (snake.getOrientacionDeLaCabeza() == EstadoSnake.NORTE) {
            snake.setOrientacionDeLaCabeza(EstadoSnake.ESTE);
            snake.incVecesQueSeGiroDerecha();

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.OESTE) {
            snake.setOrientacionDeLaCabeza(EstadoSnake.NORTE);
            snake.incVecesQueSeGiroDerecha();

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.SUR) {
            snake.setOrientacionDeLaCabeza(EstadoSnake.OESTE);
            snake.incVecesQueSeGiroDerecha();

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.ESTE) {
            snake.setOrientacionDeLaCabeza(EstadoSnake.SUR);
            snake.incVecesQueSeGiroDerecha();

            return snake;
        }

        return null;
    }

    @Override
    public EnvironmentState execute(AgentState ast, EnvironmentState est) {
        EstadoAmbiente estA = (EstadoAmbiente) est;
        EstadoSnake snake = (EstadoSnake) ast;

        if (snake.getOrientacionDeLaCabeza() == EstadoSnake.NORTE) {
            snake.setOrientacionDeLaCabeza(EstadoSnake.ESTE);
            estA.setOrientacionDeLaCabeza(EstadoSnake.ESTE);
            snake.incVecesQueSeGiroDerecha();

        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.OESTE) {
            snake.setOrientacionDeLaCabeza(EstadoSnake.NORTE);
            estA.setOrientacionDeLaCabeza(EstadoSnake.NORTE);
            snake.incVecesQueSeGiroDerecha();

        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.SUR) {
            snake.setOrientacionDeLaCabeza(EstadoSnake.OESTE);
            estA.setOrientacionDeLaCabeza(EstadoSnake.OESTE);
            snake.incVecesQueSeGiroDerecha();
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.ESTE) {
            snake.setOrientacionDeLaCabeza(EstadoSnake.SUR);
            estA.setOrientacionDeLaCabeza(EstadoSnake.SUR);
            snake.incVecesQueSeGiroDerecha();
        }

        return estA;
    }

    @Override
    public Double getCost() {
        return new Double(10);
    }

    @Override
    public String toString() {
        return "Girar_Derecha";
    }
}

In [40]:
public class GirarIzquierda extends SearchAction {

    @Override
    public SearchBasedAgentState execute(SearchBasedAgentState s) {
        EstadoSnake snake = (EstadoSnake) s;
        snake.incrementarCosto(this.getCost());

//		if (snake.getVecesQueSeGiro() > 4) {
//			snake.destrabaAlAgente();
//			snake.setVecesQueSeGiro(0);
//			
//			return null;
//		}

        if (snake.getOrientacionDeLaCabeza() == EstadoSnake.NORTE) {
            snake.incVecesQueSeGiro();

            snake.setOrientacionDeLaCabeza(EstadoSnake.OESTE);

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.OESTE) {
            snake.incVecesQueSeGiro();

            snake.setOrientacionDeLaCabeza(EstadoSnake.SUR);

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.SUR) {
            snake.incVecesQueSeGiro();

            snake.setOrientacionDeLaCabeza(EstadoSnake.ESTE);

            return snake;
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.ESTE) {
            snake.incVecesQueSeGiro();

            snake.setOrientacionDeLaCabeza(EstadoSnake.NORTE);

            return snake;
        }

        return null;
    }

    @Override
    public EnvironmentState execute(AgentState ast, EnvironmentState est) {
        EstadoAmbiente estA = (EstadoAmbiente) est;
        EstadoSnake snake = (EstadoSnake) ast;

        if (snake.getOrientacionDeLaCabeza() == EstadoSnake.NORTE) {
            snake.incVecesQueSeGiro();

            snake.setOrientacionDeLaCabeza(EstadoSnake.OESTE);
            estA.setOrientacionDeLaCabeza(EstadoSnake.OESTE);
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.OESTE) {
            snake.incVecesQueSeGiro();

            snake.setOrientacionDeLaCabeza(EstadoSnake.SUR);
            estA.setOrientacionDeLaCabeza(EstadoSnake.SUR);
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.SUR) {
            snake.incVecesQueSeGiro();

            snake.setOrientacionDeLaCabeza(EstadoSnake.ESTE);
            estA.setOrientacionDeLaCabeza(EstadoSnake.ESTE);
        } else if (snake.getOrientacionDeLaCabeza() == EstadoSnake.ESTE) {
            snake.incVecesQueSeGiro();

            snake.setOrientacionDeLaCabeza(EstadoSnake.NORTE);
            estA.setOrientacionDeLaCabeza(EstadoSnake.NORTE);
        }

        return estA;
    }

    @Override
    public Double getCost() {
        return new Double(10);
    }

    @Override
    public String toString() {
        return "Girar_Izquierda";
    }
}

## Snake Agent

In [41]:
public class AgenteSnake extends SearchBasedAgent {

    private Calculador calculador;

    @Override
    public Action selectAction() {
        // Instanciación la estrategia de búsqueda primero en profundidad.-
//		DepthFirstSearch estrategiaBusqueda = new DepthFirstSearch();

        /**
         * Ejemplos de instanciación de otras estrategias de búsqueda.-
         * 
         * // Instanciación de la estrategia primero en amplitud.-
         * BreathFirstSearch estrategiaBusqueda = new BreathFirstSearch();
         * 
         * // Instanciación de la estrategia de costo uniforme.-
         * IStepCostFunction costo = new FuncionCosto();
         * UniformCostSearch estrategiaBusqueda = new UniformCostSearch(costo);
         *///		IStepCostFunction costo = new FuncionCosto();
//		UniformCostSearch estrategiaBusqueda = new UniformCostSearch(costo);
        DepthFirstSearch estrategiaBusqueda = new DepthFirstSearch();

        // Instancia un proceso de búsqueda indicando como parámetro la estrategia a utilizar.-
        Search busqueda = new Search(estrategiaBusqueda);

        // Indica que el árbol de búsqueda debe ser mostrado e formato XML.-
        busqueda.setVisibleTree(Search.XML_TREE);

        // Le indica al Solver el proceso de búsqueda que debe ejecutar.- 
        this.setSolver(busqueda);

        // Se ejecuta el proceso de selección de la acción más adecuada.-
        Action accionSeleccionada = null;
        try {
            accionSeleccionada = this.getSolver().solve(new Object[]{this.getProblem()});
        } catch (Exception ex) {
            Logger.getLogger(AgenteSnake.class.getName()).log(Level.SEVERE, null, ex);
        }

        if (accionSeleccionada.toString().equals("Avanzar")) {
            this.calculador.reportarAccion(Calculador.AVANZAR);
        } else if (accionSeleccionada.toString().equals("Comer")) {
            this.calculador.reportarAccion(Calculador.COMER);
        } else {
            this.calculador.reportarAccion(Calculador.GIRAR);        // Retorna la acción seleccionada.-
        }
        
        return accionSeleccionada;
    }

    public AgenteSnake(Calculador calculador) {
        // Instancia la meta del Snake.-
        MetaSnake meta = new MetaSnake();
        // Instancia el estado inicial del Snake.-
        EstadoSnake estado = new EstadoSnake();

        this.setAgentState(estado);

        // Se generan las instancias de los operadores del Snake.-
        Vector<SearchAction> operadores = new Vector<SearchAction>();
        operadores.addElement(new Comer());
        operadores.addElement(new Avanzar());
        operadores.addElement(new GirarIzquierda());
        operadores.addElement(new GirarDerecha());

        // Se inicializa y asigna el problema inicial que debe resolver el Snake.-
        EstadoSnake estSnake = (EstadoSnake) this.getAgentState();
        Problem problema = new Problem(meta, estSnake, operadores);
        this.setProblem(problema);

        this.calculador = calculador;
    }

	@Override
	public void see(Perception p) {
		this.getAgentState().updateState(p);
	}
}

## Snake perceptions

In [42]:
public class PercepcionSnake extends Perception {

    public static int NO_HAY_NADA = 0;
    public static int LIMITE_DEL_MUNDO = 1;
    public static int COMIDA = 2;
    public static int PARTE_DEL_AGENTE = 3;
    public static int DESCONOCIDO = -1;
    private int sensorOeste;
    private int sensorNorte;
    private int sensorEste;
    private int sensorSur;

    public PercepcionSnake() {
    }

    public PercepcionSnake(Agent agent, Environment environment) {
        super(agent, environment);
    }

    @Override
    public void initPerception(Agent agent, Environment environment) {
        AmbienteSnake ambiente = (AmbienteSnake) environment;

        this.setSensorNorte(ambiente.getNorte());
        this.setSensorOeste(ambiente.getOeste());
        this.setSensorEste(ambiente.getEste());
        this.setSensorSur(ambiente.getSur());
    }

    public int getSensorSur() {
        return sensorSur;
    }

    public void setSensorSur(int sensorAbajo) {
        this.sensorSur = sensorAbajo;
    }

    public int getSensorNorte() {
        return sensorNorte;
    }

    public void setSensorNorte(int sensorArriba) {
        this.sensorNorte = sensorArriba;
    }

    public int getSensorEste() {
        return sensorEste;
    }

    public void setSensorEste(int sensorDerecha) {
        this.sensorEste = sensorDerecha;
    }

    public int getSensorOeste() {
        return sensorOeste;
    }

    public void setSensorOeste(int sensorIzquierda) {
        this.sensorOeste = sensorIzquierda;
    }
}

# Environment Definition

## Environment State

In [43]:
public class EstadoAmbiente extends EnvironmentState {

    private int[][] mundo;
    private int orientacionDeLaCabeza;
    private Vector<Point> composicionDelAgente;
    private Calculador calculador;

    EstadoAmbiente(int[][] m, int orientacion, Vector<Point> composicionDelAgente) {
        this.mundo = m;
        this.orientacionDeLaCabeza = orientacion;
        this.composicionDelAgente = composicionDelAgente;
    }

    EstadoAmbiente(Calculador calculador) {
        this.calculador = calculador;
        this.initState();
    }

    public int[][] getMundo() {
        return mundo;
    }

    public void setMundo(int[][] mundo) {
        this.mundo = mundo;
    }

    public void setMundo(int fil, int col, int valor) {
        this.mundo[fil][col] = valor;
    }

    public Object clone() {
        int[][] nuevoMundo = new int[mundo.length][mundo.length];

        for (int fil = 0; fil < this.mundo.length; fil++) {
            for (int col = 0; col < this.mundo.length; col++) {
                nuevoMundo[fil][col] = this.mundo[fil][col];
            }
        }
        EstadoAmbiente nuevoEstado = new EstadoAmbiente(nuevoMundo, this.orientacionDeLaCabeza, this.composicionDelAgente);

        return nuevoEstado;
    }

    public void initState() {
        Pair dimensiones = calculador.getDimensiones();

        this.mundo = new int[dimensiones.x()][dimensiones.y()];

        for (int fil = 0; fil < this.mundo.length; fil++) {
            for (int col = 0; col < this.mundo.length; col++) {
                this.mundo[fil][col] = PercepcionSnake.NO_HAY_NADA;
            }
        }
        Iterator itComida = calculador.getComida().iterator();

        while (itComida.hasNext()) {
            Pair comida = (Pair) itComida.next();

            this.mundo[comida.x() - 1][comida.y() - 1] = PercepcionSnake.COMIDA;
        }

        Pair posicionInicial = calculador.getPosicionInicial();
        this.mundo[posicionInicial.x() - 1][posicionInicial.y() - 1] = PercepcionSnake.PARTE_DEL_AGENTE;

        // Por defecto el agente arranca mirando al Este.
        this.orientacionDeLaCabeza = EstadoSnake.ESTE;

        this.composicionDelAgente = new Vector<Point>();
        this.composicionDelAgente.add(new Point(posicionInicial.x() - 1, posicionInicial.y() - 1));
    }

    public int getNorte() {
        if (this.getCabezaAgente().getX() == 0) {
            return PercepcionSnake.LIMITE_DEL_MUNDO;
        } else {
            return this.mundo[(int) this.getCabezaAgente().getX() - 1][(int) this.getCabezaAgente().getY()];
        }
    }

    public int getOeste() {
        if (this.getCabezaAgente().getY() == 0) {
            return PercepcionSnake.LIMITE_DEL_MUNDO;
        } else {
            return this.mundo[(int) this.getCabezaAgente().getX()][(int) this.getCabezaAgente().getY() - 1];
        }
    }

    public int getEste() {
        if (this.getCabezaAgente().getY() == this.mundo.length - 1) {
            return PercepcionSnake.LIMITE_DEL_MUNDO;
        } else {
            return this.mundo[(int) this.getCabezaAgente().getX()][(int) this.getCabezaAgente().getY() + 1];
        }
    }

    public int getSur() {
        if (this.getCabezaAgente().getX() == this.mundo.length - 1) {
            return PercepcionSnake.LIMITE_DEL_MUNDO;
        } else {
            return this.mundo[(int) this.getCabezaAgente().getX() + 1][(int) this.getCabezaAgente().getY()];
        }
    }

    private Point getCabezaAgente() {
        return this.composicionDelAgente.firstElement();
    }

    public String toString() {
        String str = "";

        str = str + "[ \n";
        for (int fil = 0; fil < this.mundo.length; fil++) {
            str = str + "[ ";
            for (int col = 0; col < this.mundo.length; col++) {
                str = str + this.mundo[fil][col] + " ";
            }
            str = str + " ]\n";
        }
        str = str + " ]";

        return str;
    }

    public boolean equals(Object obj) {

        return true;
    }

    public void moverAlNorte() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX() - 1,
                (int) this.composicionDelAgente.firstElement().getY()), 0);

        Point puntoABorrar = this.composicionDelAgente.remove(this.composicionDelAgente.size() - 1);

        this.mundo[(int) this.composicionDelAgente.firstElement().getX()][(int) this.composicionDelAgente.firstElement().getY()] = EstadoSnake.PARTE_DEL_AGENTE;
        this.mundo[(int) puntoABorrar.getX()][(int) puntoABorrar.getY()] = EstadoSnake.NO_HAY_NADA;
    }

    public void moverAlSur() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX() + 1,
                (int) this.composicionDelAgente.firstElement().getY()), 0);

        Point puntoABorrar = this.composicionDelAgente.remove(this.composicionDelAgente.size() - 1);

        this.mundo[(int) this.composicionDelAgente.firstElement().getX()][(int) this.composicionDelAgente.firstElement().getY()] = EstadoSnake.PARTE_DEL_AGENTE;
        this.mundo[(int) puntoABorrar.getX()][(int) puntoABorrar.getY()] = EstadoSnake.NO_HAY_NADA;
    }

    public void moverAlEste() {
        // TODO Ver que pasa si se choca una pared.....??
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX(),
                (int) this.composicionDelAgente.firstElement().getY() + 1), 0);

        Point puntoABorrar = this.composicionDelAgente.remove(this.composicionDelAgente.size() - 1);

        this.mundo[(int) this.composicionDelAgente.firstElement().getX()][(int) this.composicionDelAgente.firstElement().getY()] = EstadoSnake.PARTE_DEL_AGENTE;
        this.mundo[(int) puntoABorrar.getX()][(int) puntoABorrar.getY()] = EstadoSnake.NO_HAY_NADA;
    }

    public void moverAlOeste() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX(),
                (int) this.composicionDelAgente.firstElement().getY() - 1), 0);

        Point puntoABorrar = this.composicionDelAgente.remove(this.composicionDelAgente.size() - 1);

        this.mundo[(int) this.composicionDelAgente.firstElement().getX()][(int) this.composicionDelAgente.firstElement().getY()] = EstadoSnake.PARTE_DEL_AGENTE;
        this.mundo[(int) puntoABorrar.getX()][(int) puntoABorrar.getY()] = EstadoSnake.NO_HAY_NADA;
    }

    public int getOrientacionDeLaCabeza() {
        return orientacionDeLaCabeza;
    }

    public void setOrientacionDeLaCabeza(int orientacionDeLaCabeza) {
        this.orientacionDeLaCabeza = orientacionDeLaCabeza;
    }

    public void comerAlNorte() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX() - 1,
                (int) this.composicionDelAgente.firstElement().getY()), 0);

        this.mundo[(int) this.composicionDelAgente.firstElement().getX()][(int) this.composicionDelAgente.firstElement().getY()] = EstadoSnake.PARTE_DEL_AGENTE;
    }

    public void comerAlSur() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX() + 1,
                (int) this.composicionDelAgente.firstElement().getY()), 0);

        this.mundo[(int) this.composicionDelAgente.firstElement().getX()][(int) this.composicionDelAgente.firstElement().getY()] = EstadoSnake.PARTE_DEL_AGENTE;
    }

    public void comerAlEste() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX(),
                (int) this.composicionDelAgente.firstElement().getY() + 1), 0);

        this.mundo[(int) this.composicionDelAgente.firstElement().getX()][(int) this.composicionDelAgente.firstElement().getY()] = EstadoSnake.PARTE_DEL_AGENTE;
    }

    public void comerAlOeste() {
        this.composicionDelAgente.insertElementAt(new Point(
                (int) this.composicionDelAgente.firstElement().getX(),
                (int) this.composicionDelAgente.firstElement().getY() - 1), 0);

        this.mundo[(int) this.composicionDelAgente.firstElement().getX()][(int) this.composicionDelAgente.firstElement().getY()] = EstadoSnake.PARTE_DEL_AGENTE;
    }
}

## Snake environment 

In [44]:
public class AmbienteSnake extends Environment {

    private Calculador calculador;

    public AmbienteSnake(Calculador calculador) {
        this.calculador = calculador;
        this.environmentState = new EstadoAmbiente(this.calculador);
    }

    @Override
    public Perception getPercept() {
        //		 El ambiente crea una percepción que va a ser recibida por el Snake.- 
        PercepcionSnake p = new PercepcionSnake();

        // Es necesario realizar un "cast" para acceder a los métodos del agente Snake.- 
        //AgenteSnake snake = (AgenteCalculus)agent;

        // Asigna las percepciones en los sensores.-
        p.setSensorNorte(this.getNorte());
        p.setSensorOeste(this.getOeste());
        p.setSensorEste(this.getEste());
        p.setSensorSur(this.getSur());

        // Retorna la nueva percepción creada.-
        return p;
    }

    public int getNorte() {
        return ((EstadoAmbiente) this.environmentState).getNorte();
    }

    public int getOeste() {
        return ((EstadoAmbiente) this.environmentState).getOeste();
    }

    public int getEste() {
        return ((EstadoAmbiente) this.environmentState).getEste();
    }

    public int getSur() {
        return ((EstadoAmbiente) this.environmentState).getSur();
    }

    public String toString() {
        return environmentState.toString();
    }
}

# Cost function

In [45]:
public class FuncionCosto implements IStepCostFunction {

    public double calculateCost(AgentState agentState) {
        return ((EstadoSnake) agentState).getCosto();
    }

    @Override
    public double calculateCost(NTree node) {
        // TODO Auto-generated method stub
        return 0;
    }
}

# Main

In [ ]:
Calculador calculador = new Calculador();
AgenteSnake snake = new AgenteSnake(calculador);
AmbienteSnake ambiente = new AmbienteSnake(calculador);
SearchBasedAgentSimulator simu = new SearchBasedAgentSimulator(ambiente, snake);
simu.start();
System.out.println("Performance: " + calculador.getPerformance());


----------------------------------------------------
--- Search Based Simulator ---
----------------------------------------------------

Goal: 
------------------------------------
Sending perception to agent...
Perception: REPL.$JShell$52C$PercepcionSnake@44446fee
Agent State:  composicion="{(1,1)}"
 longitud="1"
 orientacion_cabeza="Este"
 esta_vivo="true"
mundo_conocido="[ 
[ * 2 *  ]
[ 0 3 0  ]
[ * 0 *  ]
 ]"
Environment: [ 
[ 0 0 2 2  ]
[ 0 2 2 0  ]
[ 0 3 0 0  ]
[ 2 0 0 0  ]
 ]
Asking the agent for an action...
Action returned: Avanzar

------------------------------------
Sending perception to agent...
Perception: REPL.$JShell$52C$PercepcionSnake@4a839916
Agent State:  composicion="{(1,2)}"
 longitud="1"
 orientacion_cabeza="Este"
 esta_vivo="true"
mundo_conocido="[ 
[ * 2 2 *  ]
[ 0 0 3 0  ]
[ * 0 0 *  ]
[ * * * *  ]
 ]"
Environment: [ 
[ 0 0 2 2  ]
[ 0 2 2 0  ]
[ 0 0 3 0  ]
[ 2 0 0 0  ]
 ]
Asking the agent for an action...
Action returned: Avanzar

----------------------------